<a href="https://colab.research.google.com/github/amelieav/pytorch-experiments/blob/main/leaguemodel_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
import os

def delete_all_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

# Delete files in each folder
#delete_all_files_in_folder('training')
#delete_all_files_in_folder('test')
#delete_all_files_in_folder('validation')

In [ ]:
import shutil
for folder in ['training', 'validation', 'test']:
    checkpoint_path = os.path.join(folder, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_path):
        shutil.rmtree(checkpoint_path)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.models.resnet import ResNet50_Weights

# Data Loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(root='/content/training', transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = datasets.ImageFolder(root='/content/validation', transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Definition
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features

# Assuming `n` classes (champions)
n_classes = len(train_dataset.classes)
model.fc = nn.Linear(num_ftrs, n_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 2. Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}")

    # 3. Validation Loop
    model.eval()
    val_running_loss = 0.0
    correct_preds = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)

        val_epoch_loss = val_running_loss / len(val_dataset)
        val_accuracy = correct_preds.double() / len(val_dataset)

    print(f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_accuracy:.4f}")


Epoch [1/10] Loss: 1.1184
Validation Loss: 1.0204, Accuracy: 0.4545
Epoch [2/10] Loss: 1.0827
Validation Loss: 1.0083, Accuracy: 0.4545
Epoch [3/10] Loss: 1.0192
Validation Loss: 0.9966, Accuracy: 0.5455
Epoch [4/10] Loss: 0.9377
Validation Loss: 0.9852, Accuracy: 0.5455
Epoch [5/10] Loss: 0.8468
Validation Loss: 0.9731, Accuracy: 0.4545
Epoch [6/10] Loss: 0.7535
Validation Loss: 0.9617, Accuracy: 0.4545
Epoch [7/10] Loss: 0.6621
Validation Loss: 0.9509, Accuracy: 0.4545
Epoch [8/10] Loss: 0.5756
Validation Loss: 0.9381, Accuracy: 0.4545
Epoch [9/10] Loss: 0.4960
Validation Loss: 0.9233, Accuracy: 0.4545
Epoch [10/10] Loss: 0.4245
Validation Loss: 0.9061, Accuracy: 0.4545


In [ ]:
test_dataset = datasets.ImageFolder(root='/content/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
test_running_loss = 0.0
correct_preds = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)

    test_loss = test_running_loss / len(test_dataset)
    test_accuracy = correct_preds.double() / len(test_dataset)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Test Loss: 1.0467, Test Accuracy: 0.4000


In [ ]:
def predict_image(image_path, model, transform, classes):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        output = model(image)
        probabilities = nn.functional.softmax(output, dim=1)[0] * 100
        _, predicted_class = torch.max(output, 1)

    print(f"Predicted class: {classes[predicted_class.item()]}")
    for i, score in enumerate(probabilities):
        print(f"{classes[i]}: {score:.2f}%")

    return classes[predicted_class.item()]

# Example usage:
# Annie image used appears most visually similar to Ahriimage_path = "/content/prediction/Annie_1.jpg"
predicted_class = predict_image(image_path, model, transform, train_dataset.classes)


Predicted class: ahri
aatrox: 30.95%
ahri: 41.07%
anivia: 27.98%
